In [7]:
! pip install --upgrade --quiet  langchain-openai tavily-python

In [ ]:
! pip install --upgrade langchain

In [ ]:
import os
import sys
import numpy as np
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains import create_retrieval_chain
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults

current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
sys.path.append(parrent_path)

from Update_Git import git_add, git_commit, git_push

file_path = os.path.join(current_path, 'Test_6.ipynb')
git_add(file_path)
git_commit('Updated test 6')
git_push('main')

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_path, ".env")
load_dotenv(dotenv_path=env_path)



model = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature=0.5
)

prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a friendly assisstant called gyris'),
    MessagesPlaceholder(variable_name = "chat_history"),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name = "agent_scratchpad")
])

search = TavilySearchResults()
tools = [search]

agent = create_openai_functions_agent(
    llm = model,
    prompt = prompt,
    tools = tools
)

agentExecutor = AgentExecutor(
    agent = agent,
    tools = tools
)

def process_chat(agentExecutor, user_input, chat_history):

    response = agentExecutor.invoke({
    "input": user_input,
    "chat_history": chat_history
    })

    return response



if __name__ == "__main__":
    chat_history = []

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        response = process_chat(agentExecutor, user_input, chat_history)
        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=response))
        print("AI: ", response)


{'input': 'What is the weather condition in Copenhagen?', 'output': 'The current weather condition in Copenhagen is partly cloudy with a temperature of -0.9°C (30.4°F). The wind speed is 20.5 km/h coming from the northeast direction. The humidity is at 74% with a cloud cover of 75%. It feels like -6.5°C (20.4°F) due to the wind chill factor.'}
